# Прогноз оттока клиентов телекоммуникационной компании

Оператор связи «ТелеДом» хочет бороться с оттоком клиентов. Для этого его сотрудники начнут предлагать промокоды и специальные условия всем, кто планирует отказаться от услуг связи. Чтобы заранее находить таких пользователей, «ТелеДому» нужна модель, которая будет предсказывать, разорвет ли абонент договор. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и услугах. Наша задача — обучить на этих данных модель для прогноза оттока клиентов.

## Описание задания и данных


### Описание услуг оператора
    
«ТелеДом» предоставляет два основных типа услуг:

* стационарную телефонную связь с возможностью подключения телефона к нескольким линиям одновременно;
* подключение к интернету через DSL или оптоволоконный кабель.

Также абонентам доступен ряд дополнительных услуг:

* интернет-безопасность: антивирус и блокировка опасных сайтов;
* выделенная линия технической поддержки;
* облачное хранилище файлов для резервного копирования данных;
* стриминговое телевидение и каталог фильмов.

За услуги клиенты могут платить ежемесячно или раз в год-два. Доступны различные способы расчета и возможность получать электронные чеки.
    
    
### Описание данных
    
    
Данные хранятся в базе данных, состоящей из нескольких таблиц:

* `contract` — информация о договорах;
* `personal` — персональные данные клиентов;
* `internet` — информация об интернет-услугах;
* `phone` — информация об услугах телефонии.

Таблица `contract` содержит следующие столбцы с данными:

* `customerID` — ID абонента;
* `BeginDate` — дата начала действия договора;
* `EndDate` — дата окончания действия договора;
* `Type` — тип оплаты: раз в год-два или ежемесячно;
* `PaperlessBilling` — электронный чек;
* `PaymentMethod` — тип платежа;
* `MonthlyCharges` — расходы за месяц;
* `TotalCharges` — общие расходы абонента.

Таблица `personal` содержит следующие столбцы с данными:

* `customerID` — ID пользователя;
* `gender` — пол;
* `SeniorCitizen` — является ли абонент пенсионером;
* `Partner` — есть ли у абонента партнер;
* `Dependents` — есть ли у абонента дети.

Таблица `internet` содержит следующие столбцы с данными:

* `customerID` — ID пользователя;
* `InternetService` — тип подключения;
* `OnlineSecurity` — блокировка опасных сайтов;
* `OnlineBackup` — облачное хранилище файлов для резервного копирования данных;
* `DeviceProtection` — антивирус;
* `TechSupport` — выделенная линия технической поддержки;
* `StreamingTV` — стриминговое телевидение;
* `StreamingMovies` — каталог фильмов.

Таблица `phone` содержит следующие столбцы с данными:

* `customerID` — ID пользователя;
* `MultipleLines` — подключение телефона к нескольким линиям одновременно.

Информация о договорах актуальна на **01.02.2020**.

## Загрузка библиотек и данных. Предобработка данных

Перед непосредсвенно импортами обновим или установим библиотеку `phik`, обновим библиотеки `plotly` и `scikit-learn`.

In [1]:
try:
    !pip install -U phik -q
except ModuleNotFoundError:
    !pip install phik -q

!pip install -U plotly -q
!pip install -U scikit-learn -q

Прежде чем рабодать с данными, импортируем необходимые инструменты.

In [2]:
import numpy as np
import pandas as pd

from phik import phik_matrix
from plotly import express as px
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier
)

from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)

from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split
)

from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler
)

from sqlalchemy import create_engine

CURRENT_DATE = "2020-02-01"
RANDOM_STATE=180225

Загрузим файл с базой данных в рабочую папку с проектом.

In [3]:
!wget https://code.s3.yandex.net/data-scientist/ds-plus-final.db

--2025-02-21 17:28:31--  https://code.s3.yandex.net/data-scientist/ds-plus-final.db
Resolving code.s3.yandex.net (code.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to code.s3.yandex.net (code.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3588096 (3.4M) [application/octet-stream]
Saving to: ‘ds-plus-final.db.1’

ds-plus-final.db.1  100%[===================>]   3.42M  2.66MB/s    in 1.3s    

2025-02-21 17:28:33 (2.66 MB/s) - ‘ds-plus-final.db.1’ saved [3588096/3588096]



Создадим подключение к базе данных.

In [4]:
path_to_db = "ds-plus-final.db"
engine = create_engine(f"sqlite:///{path_to_db}")

Посмотрим на таблицу `contract`.

In [5]:
df_contract = pd.read_sql("SELECT * FROM contract", engine)

print(
    f"Размерность: {df_contract.shape[0]:,} строки; {df_contract.shape[1]} столбцов.\n"
    f"Количество дубликатов: {df_contract.duplicated().sum()}.\n\n"
    f"Пропуски\n\n{df_contract.isna().sum()}"
)

display(df_contract.head())

Размерность: 7,043 строки; 8 столбцов.
Количество дубликатов: 0.

Пропуски

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,31.04
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,2071.84
2,3668-QPYBK,2019-10-01,No,Month-to-month,Yes,Mailed check,53.85,226.17
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.3,1960.6
4,9237-HQITU,2019-09-01,No,Month-to-month,Yes,Electronic check,70.7,353.5


Таблица включает **7 043** записи и **8** столбцов. В данных отсутствуют пропущенные значения и дубликаты. Названия столбцов соответствуют их описанию.  
Столбцы `BeginaDate` и `EndDate` содержат информацию о датах заключения и окончания договора. Однако в `EndDate` встречается значение **No**, что означает, что на **01.02.2020** договор остается активным. Эти текстовые «пропуски» потребуется обработать.  
Столбцы `Type`, `PaperlessBilling` и `PaymentMethod` представляют категориальные данные, а `MonthlyCharges` и `TotalCharges` — количественные. При загрузке данных все столбцы автоматически получили тип `object`.  

Теперь рассмотрим таблицу `internet`.

In [6]:
df_internet = pd.read_sql("SELECT * FROM internet", engine)

print(
    f"Размерность: {df_internet.shape[0]:,} строк; {df_internet.shape[1]} столбцов.\n"
    f"Количество дубликатов: {df_internet.duplicated().sum()}.\n\n"
    f"Пропуски\n\n{df_internet.isna().sum()}"
)

display(df_internet.head())

Размерность: 5,517 строк; 8 столбцов.
Количество дубликатов: 0.

Пропуски

customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


Таблица включает **5 517** строк и **8** столбцов. Количество записей меньше, чем в предыдущей таблице, что может означать, что некоторые абоненты пользуются только стационарной телефонной связью.  
В данных отсутствуют пропуски и дубликаты. Все столбцы, кроме `customerID`, содержат категориальные данные. При загрузке все столбцы автоматически получили тип `object`.  

Теперь рассмотрим таблицу `personal`.


In [7]:
df_personal = pd.read_sql("SELECT * FROM personal", engine)

print(
    f"Размерность: {df_personal.shape[0]:,} строки; {df_personal.shape[1]} столбцов.\n"
    f"Количество дубликатов: {df_personal.duplicated().sum()}.\n\n"
    f"Пропуски\n\n{df_personal.isna().sum()}"
)

display(df_personal.head())

Размерность: 7,043 строки; 5 столбцов.
Количество дубликатов: 0.

Пропуски

customerID       0
gender           0
SeniorCitizen    0
Partner          0
Dependents       0
dtype: int64


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


Таблица включает **7 043** строки и **5** столбцов. В данных отсутствуют пропуски и дубликаты. Названия столбцов соответствуют их описанию.  
Все столбцы, кроме `customerID`, содержат категориальные данные. При загрузке данных их тип автоматически был определен как `object`.  

Теперь рассмотрим таблицу `phone`.

In [8]:
df_phone = pd.read_sql("SELECT * FROM phone", engine)

print(
    f"Размерность: {df_phone.shape[0]:,} строка; {df_phone.shape[1]} столбцов.\n"
    f"Количество дубликатов: {df_phone.duplicated().sum()}.\n\n"
    f"Пропуски\n\n{df_phone.isna().sum()}"
)

display(df_phone.head())

Размерность: 6,361 строка; 2 столбцов.
Количество дубликатов: 0.

Пропуски

CustomerId       0
MultipleLines    0
dtype: int64


,CustomerId,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


Таблица включает **6 361** строку и **2** столбца. В данных отсутствуют пропуски и дубликаты. Названия столбцов соответствуют их описанию.  
Как и в таблице `internet`, количество строк здесь меньше, чем в `contract`. Это может указывать на то, что некоторые абоненты не пользуются услугами стационарной телефонной связи.  
Столбец `MultipleLines` содержит категориальные данные. При загрузке его тип данных автоматически был определен как `object`.

Объединим все таблицы в один датафрейм, приведем названия столбцов к нужному формату, установим `customerID` в качестве индекса и выведем информацию о полученной таблице.  

In [9]:
df = df_personal.merge(
    df_contract,
    on="customerID"
).merge(
    df_internet,
    how="left",
    on="customerID"
).merge(
    df_phone.rename(columns={"CustomerId": "customerID"}),
    how="left",
    on="customerID"
)

df.rename(
    columns={
        "customerID": "id",
        "SeniorCitizen": "senior",
        "Partner": "partner",
        "Dependents": "dependents",
        "BeginDate": "start_date",
        "EndDate": "end_date",
        "Type": "subscription_type",
        "PaperlessBilling": "paperless",
        "PaymentMethod": "payment_method",
        "MonthlyCharges": "monthly_charges",
        "TotalCharges": "total_charges",
        "InternetService": "connection_type",
        "OnlineSecurity": "security",
        "OnlineBackup": "backup",
        "DeviceProtection": "antivirus",
        "TechSupport": "support",
        "StreamingTV": "tv",
        "StreamingMovies": "movies",
        "MultipleLines": "multiple_lines"
    },
    inplace=True
)

df.set_index("id", inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gender             7043 non-null   object
 1   senior             7043 non-null   object
 2   partner            7043 non-null   object
 3   dependents         7043 non-null   object
 4   start_date         7043 non-null   object
 5   end_date           7043 non-null   object
 6   subscription_type  7043 non-null   object
 7   paperless          7043 non-null   object
 8   payment_method     7043 non-null   object
 9   monthly_charges    7043 non-null   object
 10  total_charges      7043 non-null   object
 11  connection_type    5517 non-null   object
 12  security           5517 non-null   object
 13  backup             5517 non-null   object
 14  antivirus          5517 non-null   object
 15  support            5517 non-null   object
 16  tv                 5517 non-null

Необходимо обработать пропуски и изменить типы данных в некоторых столбцах.

Большинство столбцов содержат категориальные данные (`category`). Столбцы `start_date` и `end_date` следует преобразовать в формат даты (`datetime64[ns]`), а столбцы `monthly_charges` и `total_charges` — в числовой формат с плавающей точкой (`float16`).

Кроме того, мы решим вопрос с пропусками. Для всех категориальных столбцов, за исключением `connection_type` и `multiple_lines`, пропуски будут заполнены значением **No**. В столбце `connection_type` пропуски заменим на **No internet service**, а в столбце `multiple_lines` — на **No phone service**. Это добавит третью категорию в соответствующие столбцы.

В столбце `senior` для обозначения категории используются значения **0** и **1**. Заменим их на **No** и **Yes**, как это сделано в других столбцах.

В столбце `total_charges` при попытке сменить тип данных также обнаружились пропуски (отсутствие значения).

In [10]:
df[df["total_charges"] == " "]

,gender,senior,partner,dependents,start_date,end_date,subscription_type,paperless,payment_method,monthly_charges,total_charges,connection_type,security,backup,antivirus,support,tv,movies,multiple_lines
id,,,,,,,,,,,,,,,,,,,
4472-LVYGI,Female,0,Yes,Yes,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,,DSL,Yes,No,Yes,Yes,Yes,No,NaN
3115-CZMZD,Male,0,No,Yes,2020-02-01,No,Two year,No,Mailed check,20.25,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
5709-LVOEQ,Female,0,Yes,Yes,2020-02-01,No,Two year,No,Mailed check,80.85,,DSL,Yes,Yes,Yes,No,Yes,Yes,No
4367-NUYAO,Male,0,Yes,Yes,2020-02-01,No,Two year,No,Mailed check,25.75,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
1371-DWPAZ,Female,0,Yes,Yes,2020-02-01,No,Two year,No,Credit card (automatic),56.05,,DSL,Yes,Yes,Yes,Yes,Yes,No,NaN
7644-OMVMY,Male,0,Yes,Yes,2020-02-01,No,Two year,No,Mailed check,19.85,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3213-VVOLG,Male,0,Yes,Yes,2020-02-01,No,Two year,No,Mailed check,25.35,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2520-SGTTA,Female,0,Yes,Yes,2020-02-01,No,Two year,No,Mailed check,20.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2923-ARZLG,Male,0,Yes,Yes,2020-02-01,No,One year,Yes,Mailed check,19.7,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


Как показала проверка, пропуски возникают только у клиентов, заключивших договор **01.02.2020**, то есть у новых абонентов, которые еще не успели произвести платежи. Мы заменим эти пропуски на **0**.

In [11]:
df.loc[df["start_date"] == "2020-02-01", "total_charges"] = 0

В рамках задания нам рекомендовано создать дополнительные признаки. Мы создадим признаки, которые будут отражать продолжительность контракта абонента в днях `contract_duration`, а также информацию о том, остался ли клиент или ушел `client_left`. Последний признак станет нашей целевой переменной.

Теперь напишем вспомогательные функции.

In [12]:
def contract_duration(_):
    return (_["end_date"] - _["start_date"]) // np.timedelta64(1, "D")

def client_left(_):
    if _["end_date"] == pd.to_datetime("2020-02-01", format="%Y-%m-%d"):
        return "No"
    return "Yes"

In [13]:
for column in ["start_date", "end_date"]:
    df[column] = df[column].replace("No", CURRENT_DATE)
    df[column] = pd.to_datetime(df[column], format="%Y-%m-%d")

df["client_left"] = df.apply(client_left, axis=1)
df["client_left"] = df["client_left"].astype("category")

df["contract_duration"] = df.apply(contract_duration, axis=1)
df["contract_duration"] = df["contract_duration"].astype("uint16")

for column in [
    "gender", "senior", "partner",
    "dependents", "subscription_type", "paperless",
    "payment_method", "connection_type", "security",
    "backup", "antivirus", "support",
    "tv", "movies", "multiple_lines"]:
    if column == "senior":
        df[column] = df[column].replace(["0", "1"], ["No", "Yes"])
    if column == "connection_type":
        df[column] = df[column].fillna("No internet service")
    if column == "multiple_lines":
        df[column] = df[column].fillna("No phone service")
    else:
        df[column] = df[column].fillna("No")
    df[column] = df[column].astype("category")

for column in ["monthly_charges", "total_charges"]:
    df[column] = df[column].replace(" ", 0)
    df[column] = df[column].astype("float16")

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   gender             7043 non-null   category      
 1   senior             7043 non-null   category      
 2   partner            7043 non-null   category      
 3   dependents         7043 non-null   category      
 4   start_date         7043 non-null   datetime64[ns]
 5   end_date           7043 non-null   datetime64[ns]
 6   subscription_type  7043 non-null   category      
 7   paperless          7043 non-null   category      
 8   payment_method     7043 non-null   category      
 9   monthly_charges    7043 non-null   float16       
 10  total_charges      7043 non-null   float16       
 11  connection_type    7043 non-null   category      
 12  security           7043 non-null   category      
 13  backup             7043 non-null   category      
 14

Мы загрузили данные, обновили и установили необходимые библиотеки, подключили нужные модули, задали константы `CURRENT_DATE` и `RANDOM_STATE` и установили соединение с базой данных. Затем мы извлекли данные из четырех таблиц в `pandas` и ознакомились с ними. Пропусков и дубликатов в данных не было найдено, и мы объединили все четыре таблицы в один датафрейм. Далее мы переименовали столбцы, приведя их к нижнему регистру с подчеркиваниями, а также изменили типы столбцов с `object` на `category`, `datetime64[ns]` и `float16`, заполнив пропуски в данных. В заключение мы создали новые признаки `contract_duration` и целевой `client_left`.


## Исследовательский анализ данных

В рамках задания нам предложено создать дополнительные признаки. Мы создадим признаки, которые будут показывать продолжительность контракта абонента в днях `contract_duration` и информацию о том, остался ли клиент или ушел `client_left`. Этот последний признак станет нашей целевой переменной.

Зададим функции графиков.

In [14]:
client_left_category = {
    "No": "Клиент остался",
    "Yes": "Клиент ушел"
}

def plot_hist(
    column,
    name,
    xname,
    ticks_names=None,
    ticks=[0, 1],
    mode="relative",
    lumpen=None,
    template="Количество: %{y}",
    text=True,
    norm=None,
    hover=False,
    yname="Количество"
):
    fig = px.histogram(
        df,
        x=column,  
        color="client_left", 
        title="Распределение ушедших и оставшихся клиентов " + name,
        barmode=mode,
        marginal=lumpen, 
        histnorm=norm, 
        text_auto=text 
    )

    fig.update_layout(
        title_x=.5,  
        xaxis_title=xname, 
        yaxis_title=yname, 
        showlegend=True,  
        legend_title=None,
        hovermode=hover  
    )

    if mode == "relative":
        fig.update_layout(
            xaxis=dict(
                tickmode="array",
                tickvals=ticks,
                ticktext=ticks_names
            )
        )

    fig.update_traces(hovertemplate=template) 
    fig.for_each_trace(lambda t: t.update(name=client_left_category[t.name]))  
    fig.show()

Посмотрим на признак `gender`.

In [15]:
plot_hist(
    "gender",
    "по признаку пола",
    "Пол клиента",
    ["Женщина", "Мужчина"]
)

Среди действующих клиентов компании **2 960** — женщины, а **2 982** — мужчины. Является очевидным, что пол не оказывает влияния на вероятность расторжения договора (**573** мужчины против **528** женщин).

Теперь рассмотрим признак `senior`.


In [16]:
plot_hist(
    "senior",
    "по признаку возраста",
    "Пенсионер",
    ["Нет", "Да"]
)

Среди клиентов компании **910** — пенсионеры, а **5 032** — люди другого возраста. При этом отказ от услуг компании более часто происходит среди пенсионеров, чем среди молодых клиентов: более **20 %** пенсионеров (**232** человека) против менее **15 %** в других возрастных группах (**869** человек). Возможно, для пожилых клиентов компании стоит предложить скидки на услуги, но нужно предусмотреть меры, чтобы избежать злоупотреблений, например, когда договор оформляется на пожилого родственника, а услуги используют его дети или внуки.

Теперь рассмотрим признак `partner`.


In [ ]:
plot_hist(
    "partner",
    "по признаку наличия партнера",
    "Наличие партнера",
    ["Да", "Нет"]
)

Большинство клиентов компании не состоит в партнерских отношениях **3 259** против **2 683**, однако среди тех, кто расторг договор, больше людей, состоящих в браке или партнерских отношениях (**719** против **382** среди одиноких). Возможно, в домохозяйствах с несколькими членами существуют особые требования к услугам связи, отличные от требований одиноких клиентов. Для выяснения этих потребностей целесообразно провести опрос среди семейных пар. Это может помочь снизить отток клиентов.

Теперь рассмотрим признак `dependents`.


In [ ]:
plot_hist(
    "dependents",
    "по признаку наличия детей",
    "Наличие детей",
    ["Нет", "Да"]
)

У **4 200** клиентов нет детей, а у **1 742** — есть. При этом клиенты с детьми **368** или более **17 %** расторгают договор чаще, чем клиенты без детей **733** или менее **15 %**. Так как, согласно проведенному корреляционному анализу, наличие партнера и детей взаимосвязаны, можно порекомендовать оператору связи учитывать этот фактор при анкетировании и опросах клиентов. Вопрос о наличии детей следует включить в опросы, чтобы понять потребности семейных клиентов. Возможно, для семей с детьми стоит разработать дополнительные услуги или специальные тарифы.

Теперь рассмотрим признак `subscription_type`.


In [19]:
plot_hist(
    "subscription_type",
    "по признаку типа оплаты подписки",
    "Тип оплаты подписки",
    ["Ежемесячно", "Ежегодно", "Раз в два года"],
    [0, 1, 2]
)

Среди текущих клиентов компании **3 468** оплачивают услуги ежемесячно, **1 160** — раз в год, а **1 314** — раз в два года. В абсолютных цифрах чаще всего расторгают договор клиенты с ежемесячной оплатой **407**, затем — с двухлетним периодом оплаты **381**, и реже всего уходят клиенты с ежегодной оплатой (**313**). Однако в процентном соотношении картина иная: ушли **10 %** клиентов с помесячной оплатой, более **21 %** с годовой оплатой и более **22 %** с оплатой раз в два года. Это говорит о том, что спустя определенное время клиенты с редкими платежами чаще отказываются от услуг оператора. 

Необходимо выяснить, связано ли это с качеством предоставляемых услуг, ожиданиями клиентов по скидкам за долгосрочное сотрудничество, либо с тем, что оператор не учитывает снижение цен у конкурентов, тогда как тарифы «ТелеДома» остаются неизменными.

Теперь рассмотрим признак `paperless`.


In [20]:
plot_hist(
    "paperless",
    "по признаку типа чека",
    "Тип чека",
    ["Электронный", "Бумажный"]
)

Можно было бы предположить, что клиенты, выбирающие электронные чеки, чаще расторгают договоры, чем те, кто получают бумажные. Однако нам недостаточно данных, чтобы выявить точную закономерность. Например, можно выдвинуть гипотезу, что бумажными чеками чаще пользуются пожилые клиенты, поскольку это привычный для них формат. Но при этом ранее мы установили, что пенсионеры чаще отказываются от услуг, а в случае с чеками ситуация выглядит наоборот. 

Таким образом, делать однозначные выводы преждевременно, поэтому мы просто зафиксируем данные. Среди текущих клиентов **3 450** получают электронные чеки, а **2 492** — бумажные. Среди ушедших клиентов **721** использовали электронные чеки, а **380** — бумажные.

Теперь рассмотрим признак `payment_method`.


In [21]:
plot_hist(
    "payment_method",
    "по признаку типа оплаты",
    "Тип оплаты",
    ["Электронный чек", "Чек по почте", "Банковский перевод (авто)", "Карта (авто)"],
    [0, 1, 2, 3]
)

Большинство клиентов выбирают оплату чеком — будь то электронный **2 015** или бумажный **1 495**. Чуть меньшее количество абонентов предпочитают автоматические платежи: **1 227** используют банковский перевод, а **1 205** — списание с карты.

Среди расторгнувших договоры **350** оплачивали услуги электронными чеками, **117** — бумажными, **317** пользовались автоматическим банковским переводом, и столько же — автоматическим списанием с карты.

Исходя из этих данных, можно предположить, что они собраны в США или Европе, где оплата чеками исторически более популярна. Интересно, что клиенты, использующие чеки, в среднем остаются с оператором дольше, чем те, кто настроил автоматические платежи. Возможно, те, у кого активны автоматические списания, реже задумываются об оплате, но при этом отслеживают рынок и при появлении более выгодного предложения уходят к конкурентам.

Далее рассмотрим признак `monthly_charges`.


In [22]:
plot_hist(
    "monthly_charges",
    "по признаку размера ежемесячных платежей",
    "Размер ежемесячного платежа",
    mode="overlay",
    lumpen="box",
    template="Размер ежемесячного платежа: %{x}<br>Количество: %{y}",
    text=False,
    hover=None,
    norm="percent",
    yname="Доля"
)

На графике видно, что минимальные и максимальные платежи ушедших клиентов **18,40625** и **118,75** и оставшихся **18,25** и **118,625** практически идентичны. Однако медианный платеж у расторгнувших договор был выше **84,1875** по сравнению с текущими клиентами **69,1875**.

Ранее мы предполагали, что клиенты могли уйти, найдя более выгодные тарифы у конкурентов. Данный график подтверждает эту гипотезу: в диапазоне платежей **18–21,99** доминируют текущие клиенты, в промежутке **22–81,99** их количество примерно одинаково, а начиная с **82** и выше, уже преобладают ушедшие клиенты.

Далее рассмотрим признак `total_charges`.


In [23]:
plot_hist(
    "total_charges",
    "по признаку размера общих платежей",
    "Размер общих платежей",
    mode="overlay",
    lumpen="box",
    template="Размер общих платежей: %{x}<br>Количество: %{y}",
    text=False,
    hover=None,
    norm="percent",
    yname="Доля"
)

На графике общих платежей наблюдается схожая картина с предыдущим: медианное значение у ушедших клиентов выше **2 140** по сравнению с текущими **1 193**. Максимальное значение у оставшихся клиентов достигает **9 224**, тогда как у ушедших — **7 648**. Однако стоит учитывать, что текущие клиенты в любой момент могут прекратить сотрудничество, поэтому минимальные и максимальные значения не являются ключевыми показателями.

Как будет видно на следующем графике, значительная доля текущих клиентов с общими платежами до **700** объясняется тем, что за последние **11** месяцев компания привлекла множество новых пользователей, и они пока не успели накопить значительные расходы.

Также на графике заметны выбросы, особенно среди текущих клиентов, но они выглядят естественно и не вызывают сомнений в достоверности данных.


Посмотрим на признак `connection_type`.

In [24]:
plot_hist(
    "connection_type",
    "по признаку типа интернет соединения",
    "Тип интернет соединения",
    ["Цифровая абонентская линия", "Оптоволоконный кабель", "Без интернет соединения"],
    [0, 1, 2]
)

Среди текущих клиентов компании **2 501** используют оптоволоконное подключение, **2 075** выходят в интернет через цифровую абонентскую линию *DSL*, а **1 366** вовсе не пользуются интернет-услугами оператора. 

Среди ушедших клиентов **595** использовали оптоволоконное подключение, **346** — *DSL*, а **160** не пользовались интернетом от компании. Примечательно, что доля ушедших среди пользователей оптоволоконного интернета превышает **19 %**, что значительно выше, чем в других категориях. Возможно, это связано с качеством соединения, и стоит проверить, насколько удовлетворены клиенты скоростью интернета.

Посмотрим на признак `security`.


In [25]:
plot_hist(
    "security",
    "по признаку наличия услуги блокировки сайтов",
    "Услуга блокировки сайтов",
    ["Нет", "Есть"]
)

Большая часть клиентов не подключает дополнительные услуги, но мы рассмотрим каждую из них отдельно, чтобы выявить возможные закономерности.

Среди текущих клиентов **4 338** не пользуются услугой блокировки сайтов, тогда как **1 604** подключили её. Среди ушедших клиентов **686** не пользовались этой услугой, а **415** её активировали. Видно, что среди \~ **2 000** клиентов с подключенной услугой расторгли договор около **20 %**, тогда как среди тех, кто не пользовался этой функцией, ушло менее **14 %**.

Возможно, эта услуга не оправдывает ожиданий клиентов, или же она является лишь дополнительной характеристикой ушедших пользователей, которые, как мы ранее выяснили, тратили в среднем больше, чем оставшиеся клиенты.

Посмотрим на признак `backup`.


In [26]:
plot_hist(
    "backup",
    "по признаку наличия услуги облачного хранения",
    "Услуга облачного хранения",
    ["Нет", "Есть"]
)

Похожая ситуация наблюдается и с услугой облачного хранения файлов: **4 073** текущих клиента не подключали её **541** среди ушедших, тогда как **1 869** используют этот сервис **560** среди ушедших. Доля расторгнувших договор среди пользователей данной услуги превышает **23 %**, тогда как среди тех, кто её не подключал, этот показатель составляет менее **12 %**.

Посмотрим на признак `antivirus`.


In [27]:
plot_hist(
    "antivirus",
    "по признаку наличия услуги антивируса",
    "Услуга антивируса",
    ["Нет", "Есть"]
)

Антивирусным программным обеспечением не пользуются **4 070** текущих клиентов **551** среди ушедших, а подключили его **1 872** абонента **550** среди ушедших. Доля расторгнувших договор среди пользователей данной услуги снова приближается к **23 %**, тогда как среди тех, кто её не использует, этот показатель не превышает **12 %**.

Мы наблюдаем высокий процент отказов среди пользователей всех рассмотренных дополнительных услуг. Возникает вопрос: клиенты уходят, потому что находят более выгодные предложения, или причина кроется в качестве предоставляемых услуг? Даже если мы сможем эффективно предсказать вероятность ухода клиента, стоит провести аудит качества сервисов, чтобы убедиться, что в нашем анализе учтены все возможные факторы.

Посмотрим на признак `support`.


In [28]:
plot_hist(
    "support",
    "по признаку наличия услуги выделенной поддержки",
    "Услуга выделенной поддержки",
    ["Нет", "Есть"]
)

**4 296** текущих клиентов не используют выделенную линию поддержки (**703** ушедших), в то время как **1 646** абонентов предпочитают быстро решать свои вопросы с помощью этой услуги **398** ушедших. В очередной раз доля ушедших среди пользователей услуги близка к **20 %**, тогда как среди тех, кто ею не пользуется, этот показатель составляет чуть более **14 %**.

Посмотрим на признак `tv`.


In [29]:
plot_hist(
    "tv",
    "по признаку наличия услуги стримингового ТВ",
    "Услуга стримингового ТВ",
    ["Нет", "Есть"]
)

**2 123** абонента пользуются услугой стримингового ТВ (**584** ушедших), в то время как **3 819** клиентов не используют эту услугу **517** ушедших. В целом наблюдаем схожую картину: доля ушедших среди пользователей услуги составляет более **21,5 %**, а среди тех, кто не использует услугу — около **12 %**.

Посмотрим на признак `movies`.


In [30]:
plot_hist(
    "movies",
    "по признаку наличия услуги доступа к каталогу фильмов",
    "Услуга доступа к каталогу фильмов",
    ["Нет", "Есть"]
)

Доступ к каталогу фильмов имеют **2 126** текущих клиентов **606** ушедших, а **3 816** абонентов не имеют доступа **495** ушедших. Рассматривать процентные соотношения не имеет смысла, так как результат будет похож на тот, что мы получили для стримингового ТВ.

Посмотрим на признак `multiple_lines`.


In [31]:
plot_hist(
    "multiple_lines",
    "по признаку подключения телефона к нескольким линиям",
    "Подключение телефона к нескольким линиям",
    ["Без телефонного соединения", "Нет", "Да"],
    [0, 1, 2]
)

Большинство абонентов компании не имеют подключения телефона к нескольким линиям **3 070** клиентов или не используют услуги телефонного соединения оператора **579** клиентов. Однако **2 293** абонента имеют подключение к нескольким линиям. Среди ушедших — **320**, **103** и **678** соответственно. В процентном выражении это составляет менее **10 %**, около **15 %** и около **23 %**.

Мы считаем, что уровень ухода в пределах **10–15 %** — это нормальная текучка абонентов, однако показатель **почти 23 %** — это тревожный сигнал для компании. Как упоминалось ранее, следует углубить анализ и выйти за пределы представленных данных.

Посмотрим на признак `contract_duration`.


In [32]:
plot_hist(
    "contract_duration",
    "по признаку продолжительности контракта",
    "Продолжительность контракта в месяцах",
    mode="overlay",
    lumpen="box",
    template="Продолжительность контракта: %{x}<br>Количество: %{y}",
    text=False,
    hover=None,
    norm="percent",
    yname="Доля"
)

Мы видим, что медианная продолжительность контракта ушедших абонентов составляет **915** дней, максимальная — **2 129** дней. Большинство ушедших клиентов имели контракты от **577** до **1 249** дней. Это позволяет сделать вывод, что вероятность ухода велика для абонентов, которые остаются с компанией от **1,5** до **3,5** лет.

Среди текущих клиентов медианная продолжительность контракта составляет **702** дня, максимальная — **2 314** дней. Большая часть текущих клиентов имеет контракты от **245** до **1 523** дней, что эквивалентно **0,5** года до **4** лет. Очевидно, что среди текущих абонентов есть значительное количество клиентов в «опасной» зоне, чье возможное увольнение стоит предсказать, а также предложить оператору связи рекомендации по предотвращению этого.

Посмотрим на признак `client_left`.


In [33]:
plot_hist(
    "client_left",
    "",
    "Статус клиента",
    ["Клиент остался", "Клиент ушел"],
    mode="relative"
)

В датафрейме всего содержится **7 043** записи о контрактах с клиентами. Из графика видно, что **1 101** клиент расторг договор, а **5 942** клиента на **01.02.2020** продолжали пользоваться действующими договорами.

Теперь давайте посмотрим на матрицу корреляции признаков.


In [34]:
fig = px.imshow(
    df.drop(["start_date", "end_date"], axis=1)
    .phik_matrix(interval_cols=["monthly_charges", "total_charges", "contract_duration"]),
    text_auto=".1f",
    color_continuous_scale="Viridis",
    aspect="auto",
    title=f"Матрица корреляции &#x3D5;<sub>K</sub> с тепловой картой"
)

fig.update_layout(title_x=0.5, hovermode=False)

fig.show()

Согласно матрице корреляции и шкале Чеддока, мы наблюдаем следующие результаты:
* очень сильную корреляцию между `connection_type` и `monthly_charges` (**0,9**);
* высокую корреляцию между `dependents` и `partner` (**0,7**), `total_charges` и `monthly_charges` (**0,7**), `antivirus` и `monthly_charges` (**0,7**), `tv` и `monthly_charges` (**0,8**), `movies` и `monthly_charges` (**0,8**), `movies` и `tv` (**0,7**), `multiple_lines` и `monthly_charges` (**0,7**), `multiple_lines` и `connection_type` (**0,7**), `contract_duration` и `total_charges` (**0,8**).

Во всех остальных случаях наблюдается умеренная или слабая корреляция между признаками.

Большинство выявленных корреляций можно логически интерпретировать, однако возникает вопрос: высокая корреляция между дополнительными услугами и доступом к выделенной линии технической поддержки может указывать на низкое качество услуг, сложность их настройки, или это просто свидетельствует о том, что клиенты, склонные подключать дополнительные услуги, также заинтересованы в "премиальной" поддержке?

Мы проанализировали распределение всех признаков для ушедших и оставшихся клиентов, изучили матрицу корреляции между признаками и выявили множество взаимосвязанных факторов, которые можно логически объяснить.


## Отбор признаков и подготовка выборок


В соответствии с описанием проекта, так как мы не решаем задачу временных рядов, можно исключить из анализа признаки `start_date` и `end_date`. На их основе уже были созданы новые признаки: `contract_duration` и `client_left` целевой признак.

Из исследовательского анализа мы видим, что доля мужчин и женщин среди как текущих клиентов, так и ушедших одинаковая, то есть этот признак не имеет связи с целевым. Это подтверждается и матрицей корреляции. Следовательно, признак `gender` можно исключить из дальнейшего анализа.

Кроме того, мы уберем признак `total_charges`, поскольку он является комбинацией признаков `monthly_charges` и `contract_duration`, а также имеет высокую корреляцию с другими признаками.

Рекомендуем объединить признаки `tv` и `movies` в новый общий признак `streaming`, так как, согласно матрице корреляции и результатам исследовательского анализа, эти признаки сильно схожи.


In [35]:
df["streaming"] = df[["tv", "movies"]].isin(["Yes"]).sum(axis=1)
df["streaming"] = df["streaming"].astype("category")

df.drop(
    columns=["gender", "start_date", "end_date",
             "total_charges", "tv", "movies"],
    inplace=True
)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   senior             7043 non-null   category
 1   partner            7043 non-null   category
 2   dependents         7043 non-null   category
 3   subscription_type  7043 non-null   category
 4   paperless          7043 non-null   category
 5   payment_method     7043 non-null   category
 6   monthly_charges    7043 non-null   float16 
 7   connection_type    7043 non-null   category
 8   security           7043 non-null   category
 9   backup             7043 non-null   category
 10  antivirus          7043 non-null   category
 11  support            7043 non-null   category
 12  multiple_lines     7043 non-null   category
 13  client_left        7043 non-null   category
 14  contract_duration  7043 non-null   uint16  
 15  streaming          7043 non-null   category
d

Посмотрим на обновленную матрицу корреляции, чтобы убедиться в том, что мы не сделали хуже.

In [36]:
fig = px.imshow(
    df.phik_matrix(interval_cols=["monthly_charges", "contract_duration"]),
    text_auto=".1f",
    color_continuous_scale="Viridis",
    aspect="auto",
    title=f"Матрица корреляции &#x3D5;<sub>K</sub> с тепловой картой"
)

fig.update_layout(title_x=0.5, hovermode=False)

fig.show()

Мы разделим данные на тренировочную и тестовую выборки. Тестовая выборка будет составлять **25 %** от общего объема данных (по умолчанию).


In [37]:
X_train, X_test, y_train, y_test =  train_test_split(
    df.drop(["client_left"], axis=1),
    df["client_left"],
    test_size=.25,
    random_state=RANDOM_STATE,
    stratify=df["client_left"]
)

Мы исключили преобразованные признаки и признак пола клиента, так как они не показывают связи с целевым признаком. Также был удален признак `total_charges`, который является комбинацией признаков `monthly_charges` и `contract_duration`, а признаки `tv` и `movies` были объединены в новый категориальный признак `streaming`. Данные были разделены на тренировочную и тестовую выборки в соотношении **3** к **1**.


## Обучение моделей


В рамках задания нам предстоит рассмотреть три типа моделей: дерево решений или метод случайного леса, градиентный бустинг и нейронную сеть. Поскольку метод случайного леса в прошлом показывал отличные результаты, превосходя по качеству одно дерево решений, мы решили не рассматривать отдельное дерево.

Для обучения моделей мы использовали библиотеку `LightGBM` для градиентного бустинга, а для нейронных сетей — библиотеки `PyTorch` и `TensorFlow`. В этом проекте мы решили использовать модели из библиотеки `scikit-learn`, выбрав `RandomForestClassifier`, `GradientBoostingClassifier` и `MLPClassifier`.

После преобразования данных на предыдущих этапах у нас осталось **16** признаков, из которых один является целевым. Для кодирования категориальных признаков будем использовать `OneHotEncoder` / `OrdinalEncoder`, а для кодирования целевого признака — `LabelEncoder`. Масштабирование числовых признаков выполнено с помощью `RobustScaler`.

Далее инициализируем `LabelEncoder`, произведем кодирование целевого признака и создадим пайплайн для подготовки данных и поиска лучшей модели с использованием метода `RandomizedSearchCV`.


In [38]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

cat_features = [
    "senior", "partner", "dependents",
    "subscription_type", "paperless", "payment_method",
    "connection_type", "security", "backup",
    "antivirus", "support", "multiple_lines",
    "streaming"
]

ohe_pipe = Pipeline([
    ("ohe", OneHotEncoder(drop="first", sparse_output=False))
])

baseline_mlpc_preprocessor = ColumnTransformer([
    ("ohe", ohe_pipe, cat_features),
    ("num", RobustScaler(), ["monthly_charges", "contract_duration"])
])

gbc_rfc_preprocessor = ColumnTransformer([
    ("ord", OrdinalEncoder(), cat_features),
    ("num", RobustScaler(), ["monthly_charges", "contract_duration"])
])

baseline_mlpc_pipe = Pipeline([
    ("preprocessor", baseline_mlpc_preprocessor),
    ("models", GradientBoostingClassifier())
])

gbc_rfc_pipe = Pipeline([
    ("preprocessor", gbc_rfc_preprocessor),
    ("models", GradientBoostingClassifier())
])

gbc_params = [
    {
        "models": [GradientBoostingClassifier(random_state=RANDOM_STATE)],
        "models__learning_rate": [.1, .2],
        "models__max_depth": np.arange(3, 10),
        "models__max_features": ["sqrt", "log2", None],
        "models__max_leaf_nodes": np.arange(3, 10),
        "models__min_samples_split": np.arange(2, 11),
        "models__min_samples_leaf": np.arange(1, 5),
        "models__n_estimators": np.arange(50, 301, 50)
    }
]

logreg_params = [
    {
        "models": [LogisticRegression(random_state=RANDOM_STATE)],
    }
]

mlpc_params = [
    {
        "models": [MLPClassifier(random_state=RANDOM_STATE, max_iter=1000)],
        "models__activation" :["relu", "tanh"],
        "models__alpha": [.5, 1],
        "models__hidden_layer_sizes": [(10, 10, 10), (5, 5 ,5)],
        "models__learning_rate_init" : [.01, .1, .2]
    }
]

rfc_params = [
    {
        "models": [RandomForestClassifier(random_state=RANDOM_STATE)],
        "models__class_weight": [None, "balanced"],
        "models__criterion": ["entropy", "gini", "log_loss"],
        "models__max_depth": np.arange(3, 10),
        "models__max_features": ["sqrt", "log2", None],
        "models__max_leaf_nodes": np.arange(3, 10),
        "models__min_samples_split": np.arange(2, 11),
        "models__min_samples_leaf": np.arange(1, 5),
        "models__n_estimators": np.arange(50, 301, 50)
    }
]

Для начала построим базовую модель логистической регрессии.

In [39]:
baseline_search = RandomizedSearchCV(
    baseline_mlpc_pipe,
    logreg_params,
    scoring="roc_auc",
    n_iter=1,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

baseline_search.fit(X_train, y_train)

print(f"Параметры лучшей модели:\n{baseline_search.best_params_}\n")

print(
    f"Метрика ROC-AUC базовой модели логистической регрессии на кросс-валидации: "
    f"{baseline_search.best_score_:,.4f}"
)

Параметры лучшей модели:
{'models': LogisticRegression(random_state=180225)}

Метрика ROC-AUC базовой модели логистической регрессии на кросс-валидации: 0.7581


Оценим результаты, которые покажет лучшая модель метода случайного леса.

В ходе поиска по 200 итерациям мы получили тот же результат по метрике $ROC\textrm{–}AUC$, что и при поиске по 20 итерациям.

In [40]:
rfc_search = RandomizedSearchCV(
    gbc_rfc_pipe,
    rfc_params,
    scoring="roc_auc",
    n_iter=20,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

rfc_search.fit(X_train, y_train)

print(f"Параметры лучшей модели:\n{rfc_search.best_params_}\n")

print(
    f"Метрика ROC-AUC лучшей модели метода случайного леса на кросс-валидации: "
    f"{rfc_search.best_score_:,.4f}"
)

Параметры лучшей модели:
{'models__n_estimators': 150, 'models__min_samples_split': 7, 'models__min_samples_leaf': 3, 'models__max_leaf_nodes': 7, 'models__max_features': None, 'models__max_depth': 6, 'models__criterion': 'log_loss', 'models__class_weight': None, 'models': RandomForestClassifier(random_state=180225)}

Метрика ROC-AUC лучшей модели метода случайного леса на кросс-валидации: 0.8133


Оценим результаты, которые покажет лучшая модель метода случайного леса.

В ходе поиска по 200 итерациям мы получили тот же результат по метрике $ROC\textrm{–}AUC$, что и при поиске по 20 итерациям.

In [41]:
gbc_search = RandomizedSearchCV(
    gbc_rfc_pipe,
    gbc_params,
    scoring="roc_auc",
    n_iter=20,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

gbc_search.fit(X_train, y_train)

print(f"Параметры лучшей модели:\n{gbc_search.best_params_}\n")

print(
    f"Метрика ROC-AUC лучшей модели метода градиентного бустинга на кросс-валидации: "
    f"{gbc_search.best_score_:,.4f}",
)

Параметры лучшей модели:
{'models__n_estimators': 150, 'models__min_samples_split': 10, 'models__min_samples_leaf': 2, 'models__max_leaf_nodes': 9, 'models__max_features': None, 'models__max_depth': 5, 'models__learning_rate': 0.2, 'models': GradientBoostingClassifier(random_state=180225)}

Метрика ROC-AUC лучшей модели метода градиентного бустинга на кросс-валидации: 0.9094


Посмотрим, какие результаты даст нейросетевая модель.

*Мы проверяли и иные архитектуры нейронных сетей (с **1** и с **2** скрытыми слоями и разным количеством нейронов в слоях), но они дали или худшие результаты, или аналогичные тем, что были получены, потому для ускорения процесса моделирования были оставлены только сети с небольшим количеством нейронов.*

In [42]:
mlpc_search = RandomizedSearchCV(
    baseline_mlpc_pipe,
    mlpc_params,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

mlpc_search.fit(X_train, y_train)

print(f"Параметры лучшей модели:\n{mlpc_search.best_params_}\n")

print(
    f"Метрика ROC-AUC лучшей нейросетевой модели на кросс-валидации: "
    f"{mlpc_search.best_score_:,.4f}"
)

Параметры лучшей модели:
{'models__learning_rate_init': 0.01, 'models__hidden_layer_sizes': (10, 10, 10), 'models__alpha': 0.5, 'models__activation': 'tanh', 'models': MLPClassifier(max_iter=1000, random_state=180225)}

Метрика ROC-AUC лучшей нейросетевой модели на кросс-валидации: 0.8289


В результате случайного поиска по сетке гиперпараметров были получены следующие результаты для различных моделей:

- **`LogisticRegression`** (базовая модель): Значение метрики $ROC\textrm{–}AUC$ — **0,7581**.
- **`RandomForestClassifier`**: Оптимальные параметры: `n_estimators=150`, `min_samples_split=7`, `min_samples_leaf=2`, `max_leaf_nodes=9`, `max_depth=9`, `criterion="log_loss"`. Значение метрики $ROC\textrm{–}AUC$ — **0,8133**.
- **`GradientBoostingClassifier`**: Оптимальные параметры: `n_estimators=100`, `min_samples_split=9`, `min_samples_leaf=1`, `max_leaf_nodes=8`, `max_depth=8`, `learning_rate=.2`. Значение метрики $ROC\textrm{–}AUC$ — **0,9094**.
- **`MLPClassifier`**: Оптимальные параметры: `learning_rate_init=.001`, `hidden_layer_sizes=(10, 10, 10)`, `alpha=1`, `activation="tanh"`. Значение метрики $ROC\textrm{–}AUC$ — **0,8289**.

**Итоги**: Были протестированы четыре модели: логистическая регрессия (базовая), случайный лес, градиентный бустинг и нейронная сеть. Базовая модель показала результат **0,7581**, случайный лес — **0,8133**, градиентный бустинг — **0,9094**, нейронная сеть — **0,8289**. Только модель `GradientBoostingClassifier` достигла требуемого уровня качества (**≥ 0,85**).



Мы протестировали четыре модели: базовую логистическую регрессию, случайный лес, градиентный бустинг и нейронную сеть. Логистическая регрессия показала значение метрики $ROC\textrm{–}AUC$ 0,7581, случайный лес — **0,8133**, градиентный бустинг — **0,9094**, а нейросеть — **0,8289**. Только `GradientBoostingClassifier` достиг необходимого уровня качества (**≥ 0,85**), соответствующего требованиям задания.


## Тестирование модели


На этапе обучения моделей лучшей оказалась модель градиентного бустинга. Сохраним эту модель из `gbc-search` и проведем ее тестирование.

In [43]:
best_model = gbc_search.best_estimator_

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

Построим матрицу ошибок и вычислим значение метрики $Accuracy$ на тестовой выборке.

In [44]:
fig = px.imshow(
    confusion_matrix(y_test, y_pred),
    text_auto=".0f",
    color_continuous_scale="Blues",
    aspect="auto",
    title="Матрица ошибок",
    )

fig.update_layout(
    title_x=0.5,
    hovermode=False,
    xaxis=dict(tick0=0, dtick=1),
    yaxis=dict(tick0=0, dtick=1),
    yaxis_title="Факт",
    xaxis_title="Прогноз")

fig.show()

print(
    f"Значение метрики Accuracy на тестовой выборке: "
    f"{accuracy_score(y_test, y_pred):,.4f}"
)

Значение метрики Accuracy на тестовой выборке: 0.9057


Наша модель ошибается в **\~ 10%** случаев, при этом она чаще допускает ошибку второго рода, присваивая клиенту статус оставшегося, когда он ушел.

Посмотрим на кривую ошибок.

In [45]:
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

fig = px.area(
    x=fpr, y=tpr,
    title=f"Кривая ошибок (<i>AUC</i> = {roc_auc_score(y_test, y_pred_proba):,.4f})",
    labels=dict(x="Специфичность (FPR)", y="Чувствительность (TPR)")
)
fig.add_shape(
    type="line", line=dict(dash="dash"),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_layout(title_x=0.5)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain="domain")
fig.show()

Метрика $ROC\textrm{–}AUC$ на тестовой выборке превысила установленное в задании требование и составила **0,9093**.

Используем метод `permutation_importance`, чтобы оценить вклад каждого признака в качество моделиm.

In [46]:
perm = permutation_importance(
    best_model,
    X_test,
    y_test,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

importance = pd.Series(perm["importances_mean"],
    index=best_model.feature_names_in_)

fig = px.bar(importance.sort_values(), orientation="h")

fig.update_layout(
        title_x=0.5,
        xaxis_title="Важность",
        yaxis_title="Признак",
        showlegend=False
)

fig.show()

Наибольшее влияние на предсказания модели оказали `contract_duration`, `subscription_type`, `monthly_charges`.

Как отмечалось в исследовательском анализе, вероятность ухода клиента возрастает с увеличением длительности его контракта. Также было замечено, что клиенты с редкими платежами (раз в год-два) уходят чаще, чем те, кто платит ежемесячно. Абонентская плата в среднем выше у тех, кто покидает компанию. Кроме того, подключение нескольких телефонных линий и способ оплаты также связаны с вероятностью ухода.

Далее рассмотрим график `contract_duration`.

In [47]:
plot_hist(
    "contract_duration",
    "по признаку продолжительности контракта",
    "Продолжительность контракта в месяцах",
    mode="overlay",
    lumpen="box",
    template="Продолжительность контракта: %{x}<br>Количество: %{y}",
    text=False,
    hover=None,
    norm="percent",
    yname="Доля"
)

Мы протестировали модель на тестовой выборке и получили следующие результаты: значение метрики $ROC\textrm{–}AUC$ составило **0,9093**, а значение метрики $Accuracy$ — **0,9057**. Для анализа результатов мы построили матрицу ошибок и график кривой ошибок. Исследование важности признаков выявило, что ключевыми факторами для модели являются `contract_duration`, `subscription_type`, `monthly_charges`. 

Ранее мы отмечали, что вероятность ухода клиентов наиболее высока среди абонентов, пользующихся услугами компании от **1,5** до **3,5** лет. При этом основная масса текущих клиентов сосредоточена в диапазоне от **0,5** до **4** лет, что означает значительное количество абонентов в «зоне риска». 

Мы рекомендуем оператору связи предлагать промокоды, специальные акции или тарифы клиентам, чей срок обслуживания превышает **15 месяцев**. Наибольший отток клиентов наблюдается в группе **450–499** дней, а также периодически в других группах вплоть до **1 200–1 249** дней. После этого периода отток возвращается к нормальному уровню. 

Сосредоточение усилий компании «ТелеДом» на этих группах клиентов поможет повысить лояльность и снизить отток среди колеблющихся абонентов.


## Выводы и рекомендации заказчику

### Итоги исследования

1. Мы загрузили данные в папку с проектом, установили и обновили необходимые библиотеки, импортировали их в проект. Определили константы `CURRENT_DATE` и `RANDOM_STATE`, а также создали соединение с базой данных. Затем выгрузили данные из четырех таблиц в `pandas`, изучили их и не обнаружили явных пропусков и дубликатов. Все таблицы были объединены в один датафрейм. Далее привели названия столбцов к нижнему регистру с подчеркиваниями, изменили типы данных (`object` → `category`, `datetime64[ns]`, `float16`), заполнили пропуски и добавили два новых признака: `contract_duration` и `client_left` (целевой).  

2. Мы проанализировали распределение признаков среди оставшихся и ушедших клиентов, изучили матрицу корреляции и выявили множество взаимосвязанных переменных. При этом большинство корреляций поддаются логическому объяснению.  

3. Удалены ненужные признаки, в том числе переменные с высокой корреляцией и пол клиента (так как он не влияет на целевой признак). Исключен `total_charges`, так как он является комбинацией `monthly_charges` и `contract_duration`. Объединены признаки `tv` и `movies` в новую категорию `streaming`. После этого данные были разделены на обучающую и тестовую выборки в соотношении **3:1**.  

4. Мы протестировали четыре типа моделей: логистическую регрессию (базовая модель), случайный лес, градиентный бустинг и нейросеть. Полученные результаты:  
   - Логистическая регрессия: $ROC\textrm{–}AUC$ = **0,7581**  
   - Случайный лес: **0,8133**  
   - Градиентный бустинг: **0,9094**  
   - Нейросеть: **0,8289**  
   Только `GradientBoostingClassifier` достиг требуемого качества (≥ **0,85**).  

5. Проверка модели на тестовой выборке дала следующие результаты:  
   - $ROC\textrm{–}AUC$ = **0,9093**  
   - $Accuracy$ = **0,9057**  
   Построены матрица ошибок и график кривой ошибок.  

   Анализ важности признаков показал, что наиболее значимыми являются:  
   - `contract_duration`  
   - `subscription_type`  
   - `monthly_charges`   

   Мы отметили, что высокий риск ухода наблюдается у клиентов со стажем **1,5–3,5 лет**. Основная масса клиентов находится в диапазоне **0,5–4 лет**, что означает, что многие из них подвержены риску ухода. Рекомендуем оператору связи направлять специальные предложения и промокоды клиентам, чей стаж превышает **15 месяцев**, так как именно с этого периода начинается рост оттока. Наибольший пик отказов приходится на **450–499 дней**, а затем периодически повторяется до **1 200–1 249 дней**, после чего отток снижается. Если сосредоточить усилия на удержании этой категории абонентов, можно повысить их лояльность и снизить уровень оттока.  

### Рекомендации

Чтобы сократить отток клиентов, мы рекомендуем оператору связи:  
- Провести опрос среди абонентов для уточнения их потребностей, предпочтений в ценах и услугах.  
- Оценить качество предоставляемых услуг, включая скорость интернета и надежность связи.  
- Отслеживать конкурентные предложения на рынке телекоммуникаций, так как часть ушедших клиентов могла найти более выгодные условия у других операторов.  

Эти меры помогут улучшить качество обслуживания, повысить удовлетворенность клиентов и сократить уровень оттока.
